# 🔄 ReAct (Reasoning and Acting) 프롬프팅

ReAct는 대규모 언어 모델(LLM)의 추론(Reasoning)과 행동(Acting)을 결합한 프롬프팅 패러다임입니다. 이 방식은 모델이 복잡한 작업을 수행할 때 단계별 사고 과정과 행동을 명시적으로 표현하도록 유도합니다.

## 📋 ReAct의 작동 방식

ReAct 프롬프팅은 다음과 같은 구조화된 형식으로 진행됩니다:

1. **Thought (사고)**: 문제를 분석하고 다음 단계를 계획하는 추론 과정
2. **Action (행동)**: 정보 검색, 계산 등 실제 수행하는 작업
3. **Observation (관찰)**: 행동의 결과로 얻은 정보나 피드백
4. 최종 답변에 도달할 때까지 위 과정을 반복

## ✨ ReAct의 장점

- **투명한 추론 과정**: 모델의 사고 과정을 명시적으로 보여줌
- **외부 도구 활용**: 검색, 계산기 등 외부 도구와 연계 가능
- **오류 감소**: 단계별 접근으로 복잡한 문제 해결 시 오류 감소
- **인간과 유사한 문제 해결**: 사람처럼 단계적으로 문제를 해결하는 방식 구현

## 💻 ReAct 프롬프트 예시

```
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:
```

## 🔍 ReAct 사용 사례

ReAct는 다음과 같은 복잡한 작업에 특히 유용합니다:

- 다단계 추론이 필요한 질문 답변
- 외부 정보 검색이 필요한 작업
- 수학적/논리적 문제 해결
- 복잡한 의사 결정 과정

## 🚀 구현 방법

ReAct를 구현하기 위해서는:

1. 적절한 프롬프트 템플릿 설계
2. 외부 도구와의 연동 설정 (API, 검색 엔진 등)
3. 모델의 출력을 파싱하여 다음 단계 결정
4. 최종 답변 도출까지 반복

ReAct는 Chain of Thought와 유사하지만, 추론뿐만 아니라 실제 행동을 취하고 그 결과를 관찰하는 과정이 추가되어 더 강력한 문제 해결 능력을 제공합니다.


# ReAct 샘플

In [23]:
import boto3

def invoke_bedrock_react_converse(query, model_id="anthropic.claude-3-sonnet-20240229-v1:0", region_name="us-east-1"):
    # Bedrock 클라이언트 생성
    client = boto3.client("bedrock-runtime", region_name=region_name)
    
    # ReAct 시스템 프롬프트
    react_system_prompt = """
    You are an AI assistant that follows the ReAct (Reasoning and Acting) framework.
    When answering questions, follow this format:
    Thought: Reason step-by-step about the problem
    Action: Decide what action to take
    Action Input: Provide input for the action
    Obsal Answer: Provide the final answer
    """
    
    try:
        # Converse API 호출
        response = client.converse(
            modelId=model_id,
            messages=[
                {
                    "role": "user",
                    "content": [{"text": query}]
                }
            ],
            system=[{"text": react_system_prompt}],
            inferenceConfig={
                "maxTokens": 500,
                "temperature": 0.7,
                "topP": 0.9
            }
        )
        
        # 응답 추출
        result = response["output"]["message"]["content"][0]["text"]
        return result
        
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)


In [24]:
question = "2024년 노벨 문학상 수상자는 누구인가요?"
result = invoke_bedrock_react_converse(question)
print(result)

Thought: 2024년 노벨 문학상 수상자는 아직 발표되지 않았습니다. 노벨상은 매년 10월 초에 발표되므로 2024년 수상자를 지금 알려줄 수 없습니다. 하지만 과거 수상자들의 명단과 수상 이유 등을 설명할 수 있습니다.

Action: 노벨 문학상에 대한 일반적인 정보를 제공하고, 최근 수상자들을 예시로 들어 설명한다.

Action Input: 노벨 문학상은 문학 분야에서 가장 권위 있는 상입니다. 매년 스웨덴 아카데미에서 선정하여 수여합니다. 최근 수상자로는 2022년 프랑스 작가 안니 에르노, 2021년 탄자니아 작가 압두르라즈락 구르나, 2020년 미국 시인 루이스 글룩이 있습니다. 이들은 작품 속에서 인류의 경험을 섬세하게 형상화했다는 평가를 받았습니다.

Observation Answer: 2024년 노벨 문학상 수상자는 아직 발표되지 않았습니다. 노벨 문학상은 문학 분야에서 가장 권위 있는 상으로, 매년 스웨덴 아카데미에서 선정하여 수여합니다. 최근 수상자로는 2022년 프랑스 작가 안니 에르노, 2021년 탄자니아 작가 압두르


In [44]:
import boto3
import requests
from bs4 import BeautifulSoup

# Bedrock 클라이언트 설정
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'  # 사용하는 리전에 맞게 변경하세요
)

def fetch_nobel_literature_data():
    """노벨 문학상 웹사이트에서 HTML 정보를 가져오는 함수"""
    url = "https://www.nobelprize.org/prizes/lists/all-nobel-prizes-in-literature/"
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 관련성 높은 콘텐츠만 추출
            content = soup.find('article', class_='entry entry-gutenberg')
            
            if content:
                return str(content)
            else:
                # article을 찾지 못할 경우 본문 전체 반환
                body = soup.find('body')
                return str(body) if body else response.text
        else:
            return f"웹사이트 접근 실패: 상태 코드 {response.status_code}"
    
    except Exception as e:
        return f"웹 스크래핑 중 오류 발생: {str(e)}"

def is_nobel_literature_question(message):
    """메시지가 노벨 문학상 관련 질문인지 판단하는 함수"""
    keywords = ['노벨', '문학상', 'nobel', 'literature', 'prize', '수상자', '작가']
    message_lower = message.lower()
    return any(keyword in message_lower for keyword in keywords)

def converse_with_bedrock(user_message, model_id="anthropic.claude-3-sonnet-20240229-v1:0"):
    """Bedrock Converse API를 사용하여 ReAct 방식으로 대화하는 함수"""
    
    try:
        # 노벨 문학상 관련 질문인지 확인
        if is_nobel_literature_question(user_message):
            # 1단계: 질문 분석 및 검색 계획 수립
            plan_response = bedrock_runtime.converse(
                modelId=model_id,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "text": f"""질문: {user_message}

이 질문은 노벨 문학상에 관한 것입니다. 이런 질문에 답변하기 위해 어떤 정보가 필요할지 분석하고, 
검색 계획을 세워주세요. 노벨 문학상 웹사이트를 검색할 예정입니다."""
                            }
                        ]
                    }
                ]
            )
            
            # 계획 추출
            plan = extract_text_from_response(plan_response)
            
            # 2단계: 웹 검색 수행
            print("웹 검색을 수행합니다...")
            html_data = fetch_nobel_literature_data()
            
            # 3단계: 검색 결과 분석 및 최종 답변 생성
            final_response = bedrock_runtime.converse(
                modelId=model_id,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "text": f"""# 질문
{user_message}

# 분석 및 계획
{plan}

# 검색 결과
다음은 노벨 문학상 웹사이트(https://www.nobelprize.org/prizes/lists/all-nobel-prizes-in-literature/)에서 가져온 정보입니다:

{html_data}

# 요청사항
1. 위 검색 결과를 분석하세요.
2. 질문에 답변하기 위한 관련 정보를 추출하세요.
3. 추출한 정보를 바탕으로 질문에 명확하고 정확하게 답변하세요.
4. 검색 결과에서 정보를 찾지 못했다면, 그 사실을 솔직하게 언급하세요."""
                            }
                        ]
                    }
                ]
            )
            
            return extract_text_from_response(final_response)
            
        else:
            # 일반 대화 처리
            response = bedrock_runtime.converse(
                modelId=model_id,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "text": user_message
                            }
                        ]
                    }
                ]
            )
            
            return extract_text_from_response(response)
            
    except Exception as e:
        return f"오류 발생: {str(e)}"

def extract_text_from_response(response):
    """Bedrock 응답에서 텍스트 추출"""
    try:
        if 'output' in response and 'message' in response['output']:
            message = response['output']['message']
            
            if 'content' in message and isinstance(message['content'], list):
                # 모든 텍스트 부분 결합
                result_text = ""
                for item in message['content']:
                    if isinstance(item, dict) and 'text' in item:
                        result_text += item['text']
                return result_text
            
        # 응답 구조가 예상과 다른 경우
        return "응답을 처리하는 중 오류가 발생했습니다."
    
    except Exception as e:
        return f"응답 처리 중 오류: {str(e)}"

# 예제 실행
if __name__ == "__main__":
    user_query = "최근 10년간 노벨 문학상 수상자는 누구인가요?"
    print(f"질문: {user_query}")
    response = converse_with_bedrock(user_query)
    print(f"\n응답: {response}")



질문: 최근 10년간 노벨 문학상 수상자는 누구인가요?
웹 검색을 수행합니다...

응답: 1. 검색 결과 분석:
- 노벨 문학상 웹사이트에서 최근 10년 간의 수상자 정보를 확인할 수 있습니다.
- 2013년부터 2022년까지의 수상자 이름, 국적, 수상 작품 및 업적이 나와 있습니다.

2. 관련 정보 추출:
- 2022년: 프랑스 작가 안니 에르노, "개인적 기억의 뿌리, 소외, 집단적 억압을 임상적 예리함으로 파헤친 공로"
- 2021년: 탄자니아 작가 압둘라자크 구르나, "식민지화와 난민의 운명을 관용과 동정심으로 통찰한 공로" 
- 2020년: 미국 시인 루이스 글릭, "개인의 존재를 간결한 아름다움으로 보편화한 독특한 시적 목소리"
- 2019년: 오스트리아 작가 페터 한트케, "언어 탐험에 대한 공로"
- 2018년: 폴란드 작가 올가 토카르축, "나치와 소련 체제하의 경험을 생생히 그려낸 공로"
- 2017년: 영국 작가 카즈오 이시구로, "잊혀진 역사의 왜곡된 곳에 인간의 운명을 예리하게 파헤친 공로"
- 2016년: 미국 가수 겸 작가 밥 딜런, "새로운 시적 표현을 창조한 공로"  
- 2015년: 벨라루스 작가 스베틀라나 알렉시예비치, "인간 정신의 고난과 굴욕에 대해 기념비적으로 서술한 공로"
- 2014년: 프랑스 작가 패트릭 모디아노, "현대 존재를 형이상학적 탐구로 다룬 공로"
- 2013년: 캐나다 소설가 앨리스 멍로, "현대 삶의 가장자리에 대한 단호하고 시적인 탐구 공로"

3. 답변:
최근 10년간 노벨 문학상 수상자는 다음과 같습니다:

2022년: 프랑스 작가 안니 에르노
2021년: 탄자니아 작가 압둘라자크 구르나  
2020년: 미국 시인 루이스 글릭
2019년: 오스트리아 작가 페터 한트케
2018년: 폴란드 작가 올가 토카르축
2017년: 영국 작가 카즈오 이시구로
2016년: 미국 가수 겸 작가 밥 딜런
2015년: 벨라루스 작가 스베틀라나 알렉시예비치
2014년: 프랑스 작가 패트릭 모디아노
2013년: 캐나

In [45]:
query = "2024년 노벨 문학상 수상자는 누구인가요?"
result = converse_with_bedrock(query)
print(result)

웹 검색을 수행합니다...
1. 검색 결과 분석:
- 검색 결과에는 2020년부터 2024년까지 노벨 문학상 수상자들의 정보가 제공되어 있습니다. 2024년 수상자는 한국 작가 한강(Han Kang)입니다.
- 수상자의 수상 동기와 간단한 소개가 포함되어 있습니다.
- 그러나 2025년 이후의 수상자 정보는 없습니다.

2. 관련 정보 추출:
- 2024년 노벨 문학상 수상자: 한강(Han Kang)
- 수상 동기: "치명적인 역사적 트라우마에 맞서고 인간 생명의 연약함을 드러내는 강렬한 시적 산문"

3. 답변:
2024년 노벨 문학상 수상자는 한국 작가 한강입니다. 한강은 "치명적인 역사적 트라우마에 맞서고 인간 생명의 연약함을 드러내는 강렬한 시적 산문"을 인정받아 수상하였습니다.

4. 2025년 수상자 정보는 검색 결과에 포함되어 있지 않습니다. 아직 발표되지 않았기 때문입니다.
